In [320]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [321]:
path_to_data = "C:/study/data/"

In [322]:
df = pd.read_csv(path_to_data + "features.csv")

In [323]:
df.head()

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
0,0,1430198770,7,11,5,2098,1489,20,0,0,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1,1430220345,0,42,4,1188,1033,9,0,1,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,2,1430227081,7,33,4,1319,1270,22,0,0,...,4,3,1,13.0,2130,0,0,1830,0,63
3,3,1430263531,1,29,4,1779,1056,14,0,0,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,4,1430282290,7,13,4,1431,1090,8,1,0,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [324]:
a = df.count().tolist()
value = len(df)
a = [True if i!=value else False for i in a]
cols_with_empty_values = df.columns[a].tolist()

####  1.1a Набор признаков, содержащих пропущенные значения

In [325]:
cols_with_empty_values

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

In [326]:
#df = df.fillna(0)
df.fillna(df.mean(), inplace=True)

### Gradient-boosting Classifier

Задача классификации, потому в первой части используем GBC с scikit-learn.

In [327]:
def diff(first, second):
        second = set(second)
        return [item for item in first if item not in second]

In [328]:
a = df.columns.tolist()
res_features = ['radiant_win', 'duration', 'tower_status_radiant', 'tower_status_dire','barracks_status_radiant','barracks_status_dire']
a = diff(a,res_features)
X = df[a].to_numpy()
y = df['radiant_win'].to_numpy()

In [329]:
scores = []
kf = KFold(n_splits=5,shuffle=True)
start_time = time.time()
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = GradientBoostingClassifier(n_estimators=30, verbose=False, learning_rate=0.1,random_state=1)
    clf.fit(X_train, y_train)
    clf.score(X_train, y_train)
    pred = clf.predict_proba(X_test)[:, 1]
    scores.append(roc_auc_score(y_test, pred))
cross_time = time.time() - start_time 

In [330]:
scores_different = []
scrs = []


for est in [20,25,30,35,40,45,50]:
    ind_subset = np.random.choice(97230, 20000, replace=False)
    sub_x = X[ind_subset]
    sub_y = y[ind_subset]
    kf = KFold(n_splits=5,shuffle=True)
    for train_index, test_index in kf.split(sub_x):
        X_train, X_test = sub_x[train_index], sub_x[test_index]
        y_train, y_test = sub_y[train_index], sub_y[test_index]
        clf = GradientBoostingClassifier(n_estimators=est, verbose=False, learning_rate=0.1,random_state=1)
        clf.fit(X_train, y_train)
        clf.score(X_train, y_train)
        pred = clf.predict_proba(X_test)[:, 1]
        scrs.append(roc_auc_score(y_test, pred))
    scores_different.append(np.mean(scrs))
scores_different


[0.6783935187743896,
 0.6808787229841958,
 0.6837247466748724,
 0.684553602774664,
 0.6857374053710907,
 0.6870807746043534,
 0.6875346634284734]

#### AUC-ROC score after cross-validation

In [331]:
np.mean(scores)

0.68914348482888

#### Cross-validation GBC time(sec)

In [332]:
round(cross_time,2)

313.8

### Logistic Regression

In [333]:
df.head()

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
0,0,1430198770,7,11,5,2098,1489,20,0,0,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1,1430220345,0,42,4,1188,1033,9,0,1,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,2,1430227081,7,33,4,1319,1270,22,0,0,...,4,3,1,13.0,2130,0,0,1830,0,63
3,3,1430263531,1,29,4,1779,1056,14,0,0,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,4,1430282290,7,13,4,1431,1090,8,1,0,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [334]:
a = df.count().tolist()
value = len(df)
a = [True if i!=value else False for i in a]
cols_with_empty_values = df.columns[a].tolist()

In [335]:
df.fillna(df.mean(), inplace=True)

In [336]:
a = df.columns.tolist()
res_features = ['radiant_win', 'duration', 'tower_status_radiant', 'tower_status_dire','barracks_status_radiant','barracks_status_dire']
b = diff(a,res_features)
X = df[b].to_numpy()
y = df['radiant_win'].to_numpy()

In [337]:
def get_auc_score_cross_val(name, X, y, C, random_state, kf):
    """
    name: name of classifier
    kf: Kfold
    return : cross-validation score
    """
    scores = []
    for train_index, test_index in kf.split(X):   
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #fitting 1
        #clf = eval(name+'(random_state=random_state, C=C)')
        clf = LogisticRegression(random_state=random_state, C=C)
        clf.fit(X_train, y_train)
        clf.score(X_train, y_train)
        pred = clf.predict_proba(X_test)[:, 1]
        scores.append(roc_auc_score(y_test, pred))
    return scores

In [338]:
#tuning c

C_param_range = [0.001,0.01,0.1,1,10,100,1000]
mx = 0
kf = KFold(n_splits=5, shuffle=True,random_state=1)
sc = StandardScaler()
X_std=sc.fit_transform(X)
for c in C_param_range:
    scores = get_auc_score_cross_val(name = 'LogisticRegression', X=X_std, y=y, C=c, random_state=1,kf=kf)
    res = np.mean(scores)
    if res > mx:
        mx=res
        best = c
print(best)

0.01


In [339]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
start_time = time.time()
scores = get_auc_score_cross_val(name = 'LogisticRegression', X=X, y=y, C=best, random_state=1,kf=kf)
cross_time = time.time() - start_time 

start_time = time.time()
sc = StandardScaler()
X_std=sc.fit_transform(X)
scores_std = get_auc_score_cross_val(name = 'LogisticRegression', X=X_std, y=y, C=best, random_state=1,kf=kf)
cross_time_std = time.time() - start_time 


In [340]:
print('auc_no_std:', np.mean(scores))
print('auc_std:', np.mean(scores_std))
print('time_no_std',round(cross_time,2))

auc_no_std: 0.5134743391867588
auc_std: 0.7168148415254104
time_no_std 2.72


#### 2

In [341]:
categ_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
res_features = ['radiant_win', 'duration', 'tower_status_radiant', 'tower_status_dire',
                'barracks_status_radiant','barracks_status_dire']

In [342]:
features_to_delete = categ_features+res_features

In [343]:
b = diff(a, features_to_delete)
X = df[b].to_numpy()
y = df['radiant_win'].to_numpy()

In [344]:
C_param_range = [0.001,0.01,0.1,1,10,100,1000]

kf = KFold(n_splits=5, shuffle=True,random_state=1)
mx = 0
scores = []
sc = StandardScaler()
X_std=sc.fit_transform(X)
for c in C_param_range:
    scores = get_auc_score_cross_val(name = 'LogisticRegression', X=X_std, y=y, C=c, random_state=1,kf=kf)
    res = np.mean(scores)
    if res > mx:
        mx=res
        best = c
print(best)
scores=[]

0.01


In [345]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
start_time = time.time()
scores = get_auc_score_cross_val(name = 'LogisticRegression', X=X, y=y, C=best, random_state=1,kf=kf)
cross_time = time.time() - start_time 

start_time = time.time()
sc = StandardScaler()
X_std=sc.fit_transform(X)
scores_std = get_auc_score_cross_val(name = 'LogisticRegression', X=X_std, y=y, C=best, random_state=1,kf=kf)
cross_time_std = time.time() - start_time 

In [346]:
print('auc_no_std_no_cat:', np.mean(scores))
print('auc_std_no_cat:', np.mean(scores_std))
print('time_no_std_no_cat',round(cross_time,2))

auc_no_std_no_cat: 0.5134743391867588
auc_std_no_cat: 0.7168660688991333
time_no_std_no_cat 2.35


#### 3


In [347]:
heroes = categ_features[1:]

#bag-of-words, code by me
"""
h_df = df[heroes]
values = np.unique(h_df.to_numpy())
N = max(values)
r_df = h_df[h_df.columns.tolist()[0:5]]
d_df = h_df[h_df.columns.tolist()[5:]]
df.shape[0]

#no 24, 107,108,109

X_pick = np.zeros((df.shape[0], N))

for i in range(0, df.shape[0]):
    r1_heroes = list(r_df.loc[i])
    d1_heroes = list(d_df.loc[i])
    for r in r1_heroes:
        X_pick[i,r-1]=1
    for d in d1_heroes:
        X_pick[i,d-1] = -1
"""


In [348]:
values = np.unique(h_df.to_numpy())
X_pick = np.zeros((df.shape[0], N))

for i, match_id in enumerate(df.index):
    for p in range(5):
        X_pick[i, df.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, df.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [349]:
X_pick

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.]])

#### 4

In [350]:
categ_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
res_features = ['radiant_win', 'duration', 'tower_status_radiant', 'tower_status_dire',
                'barracks_status_radiant','barracks_status_dire']

In [351]:
features_to_delete = categ_features+res_features

In [352]:
a = diff(a,features_to_delete)
X = df[a].to_numpy()
y = df['radiant_win'].to_numpy()

In [353]:
X = np.concatenate((X, X_pick), axis=1)

In [354]:
C_param_range = [0.001,0.01,0.1,1,10,100,1000]
mx = 0
kf = KFold(n_splits=5, shuffle=True,random_state=1)
sc = StandardScaler()
X_std=sc.fit_transform(X)
for c in C_param_range:
    scores = get_auc_score_cross_val(name = 'LogisticRegression', X=X_std, y=y, C=c, random_state=1,kf=kf)
    res = np.mean(scores)
    if res > mx:
        mx=res
        best = c
print(best)

0.01


In [355]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
start_time = time.time()
scores = get_auc_score_cross_val(name = 'LogisticRegression', X=X, y=y, C=best, random_state=1,kf=kf)
cross_time = time.time() - start_time 

start_time = time.time()
sc = StandardScaler()
X_std=sc.fit_transform(X)
scores_std = get_auc_score_cross_val(name = 'LogisticRegression', X=X_std, y=y, C=best, random_state=1,kf=kf)
cross_time_std = time.time() - start_time 

In [356]:
print('auc_no_std_no_cat:', np.mean(scores))
print('auc_std_no_cat:', np.mean(scores_std))
print('time_no_std_no_cat',round(cross_time,2))

auc_no_std_no_cat: 0.5134743391867588
auc_std_no_cat: 0.7520233082079777
time_no_std_no_cat 4.93


#### 6

In [357]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.8, random_state=1)

In [358]:
clf = LogisticRegression(C=best, random_state=1).fit(X_train,y_train)

In [359]:
pred = clf.predict_proba(X_test)[:, 1]
pred_matrix = np.zeros((len(pred),2))
pred_matrix[:,0] = pred
pred_matrix[:,1] = y_test
pred_matrix

array([[0.28197557, 1.        ],
       [0.49844561, 1.        ],
       [0.78008678, 1.        ],
       ...,
       [0.70133527, 1.        ],
       [0.38911061, 1.        ],
       [0.24289484, 0.        ]])

In [360]:
print('min:',round(min(pred),3),'\nmax:',round(max(pred),3))

min: 0.001 
max: 0.999


In [361]:
round(roc_auc_score(y_test, pred),2)

0.75

# Отчет

### Градиентный бустинг

####  1.1a Набор признаков, содержащих пропущенные значения

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

#### 1.1b Что могут означать пропуски в этих данных
- 'radiant_flying_courier_time' - этот признак может иметь пропущенное значение, если предмет 'flying_courier' не был приобретен командой radiant до окончания матча. 

- 'radiant_first_ward_time' - аналогично предмету 'flying_courier_time' можеть иметь ппропущенное значение, если вард не был поставлен командой radiant до окончания матча.

#### 2. Целевая переменная - "radiant_win"

#### 3-4. Градиентный бустинг и ускорение его тренировки
 Кросс-валидация для градиентного бустинга с 30 деревьями проводилась 273.64 сек. Среднее(после кросс-валидации) качество - 0.69. Потом попробовал потренировать модель для [20,25,30,35,40,45,50] деревьев. 

 После 30 деревьев качество растет незначительно, тогда как время тренировки - значительно. В большинстве случаев, пожалуй, не имеет смысла использовать больше деревьев. 

 Для ускорения тренировки такого количества классификаторов было использовано случайные подмножества длиной 20к. Так же для ускорения работы можно использовать спецификацию параметра "max_depth" ограничивая максимальную глубину деревьев.

### Логистическая регрессия

#### 1. Оценка качества логистической регрессии без подготовки данных и сравнение с GBC
Качество логистической регрессии 0.513, скорость работы намного выше - 2.18 сек. Такое низкое качество можно объяснить тем, что без стандартизации данных линейная модель сильно переобучается.Использовав стандартизацию получим качество 0.71.Стоит заметить что без стандартизации классификатор работает очень плохо, потому в дальнейшем везде использовалась стандартизация. 

#### 2. Удаление категориальных признаков
После удаления категориальных признаков почти ничего не изменилось. Объяснить это можно тем, что импользуя категориальные признаки как числовые классификатор не получает никакой полезной информации.Эти признаки в таком виде скорее выступают как шумы.

#### 3. Герои
Количество различных героев 112, но герои 24, 107,108,109 никогда не использовались исходя из этих данных

#### 4.Мешок слов
Добавление мешка слов улучшило качество. Было получено качество 0.75. Объясняется это тем, что правильный учет категориальных признаков помог улучшить качество, так как добавил полезной информации для классификатора

#### Прогнозы
    - min: 0.001 
    - max: 0.999